### Best Practices : 
1. Preprocessing and Cleaning
2. Train Test Split
3. BOW, TFIDF, Word2Vec
4. Train ML Algorithms

In [1]:
import pandas as pd
data = pd.read_csv('all_kindle_review.csv')
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [2]:
data = data[['reviewText','rating']]
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [3]:
data.shape

(12000, 2)

In [4]:
data.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [5]:
print(data['rating'].unique())
print(data['rating'].value_counts())

[3 5 4 2 1]
rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64


Preprocessing and Cleaning

In [6]:
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

In [7]:
import nltk

In [8]:
data['reviewText'] = data['reviewText'].str.lower()
data['reviewText'] = data['reviewText'].apply(lambda x : re.sub('[^a-z A-Z 0-9]+','',x))
data['reviewText'] = data['reviewText'].apply(lambda x : ' '.join(y for y in x.split() if y not in set(stopwords.words('english'))))
data['reviewText'] = data['reviewText'].apply(lambda x : re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x))
data['reviewText'] = data['reviewText'].apply(lambda x : BeautifulSoup(x, 'lxml').get_text())
data['reviewText'] = data['reviewText'].apply(lambda x : " ".join(x.split()))

In [9]:
data.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,3
1,great short read didnt want put read one sitti...,5
2,ill start saying first four books wasnt expect...,3
3,aggie angela lansbury carries pocketbooks inst...,3
4,expect type book library pleased find price right,4


In [12]:
data['rating'] = data['rating'].apply(lambda x : 0 if x <= 3 else 1)
# data['rating'] = data['rating'].apply(lambda x : 1 if x > 3)

In [15]:
data['rating'].value_counts()

rating
0    6000
1    6000
Name: count, dtype: int64

In [16]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [17]:
def lemmatize_word(words):
    return ' '.join(lemmatizer.lemmatize(word) for word in words.split())

In [18]:
data['reviewText'] = data['reviewText'].apply(lambda x : lemmatize_word(x))

In [20]:
data.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,0
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,0
3,aggie angela lansbury carry pocketbook instead...,0
4,expect type book library pleased find price right,1


In [78]:
from sklearn.model_selection import train_test_split
X,x,Y,y = train_test_split(data['reviewText'],data['rating'],test_size = 0.2, random_state = 42)

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [81]:
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X).astype(np.float32).toarray()
x_tfidf = tfidf.transform(x).astype(np.float32).toarray()

In [69]:
print(X.shape, x.shape, X_tfidf.shape, x_tfidf.shape)


(9600,) (2400,) (9600, 37363) (2400, 37363)


In [38]:
# from sklearn.naive_bayes import GaussianNB
# nb_model = GaussianNB().fit(X_tfidf,Y)

MemoryError: Unable to allocate 2.67 GiB for an array with shape (9600, 37363) and data type float64

In [82]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
batch_size = 1000
for i in range(0, len(X_tfidf), batch_size) :
    end = min(i + batch_size, len(X_tfidf))
    nb_model.partial_fit(X_tfidf[i:end], Y[i:end], classes = Y.unique())
    
    

In [85]:
batch_size = 500
y_pred = []
for i in range(0, len(x_tfidf), batch_size) :
    end = min(i + batch_size, len(x_tfidf))
    y_pred.append(nb_model.predict(x_tfidf[i:end]))

In [46]:
x_tfidf.shape

(2400, 16947)

In [57]:
# import gc
# del X,x,Y,y,tfidf,X_tfidf,x_tfidf,nb_model
# gc.collect()

NameError: name 'X' is not defined

In [89]:
# del nb_model
# import gc
# gc.collect()
y

1935     0
6494     0
1720     0
9120     0
360      0
        ..
1195     1
11877    1
5421     0
3855     1
4414     1
Name: rating, Length: 2400, dtype: int64

In [90]:
y_test = list(y)

In [91]:
y_test

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,


In [93]:
y_pred

[array([0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
        1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
        1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1,
        0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
        0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,
        0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
        0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
        0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 

In [94]:
import numpy as np
# 1. Concatenate all arrays into one
merged_array = np.concatenate(y_pred)

# 2. Convert to a Python list
y_pred = merged_array.tolist()


In [95]:
y_pred

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,


In [96]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.66      0.64      1190
           1       0.64      0.60      0.62      1210

    accuracy                           0.63      2400
   macro avg       0.63      0.63      0.63      2400
weighted avg       0.63      0.63      0.63      2400

